In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train=np.reshape(x_train, x_train.shape+(1,))
x_test=np.reshape(x_test, x_test.shape+(1,))
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(input_shape= [28, 28, 1], kernel_size= 5, filters= 8, strides= 1, 
                   activation= 'relu',  kernel_initializer= 'variance_scaling'),
        tf.keras.layers.MaxPooling2D(pool_size= [2, 2], strides= [2, 2]),
        tf.keras.layers.Conv2D(kernel_size= 5, filters= 16, strides= 1, activation= 'relu', kernel_initializer= 'variance_scaling'),
        tf.keras.layers.MaxPooling2D(pool_size= [2, 2], strides= [2, 2]),
        tf.keras.layers.Flatten(),
        #tf.keras.layers.Dense(units= 128, kernel_initializer= 'variance_scaling', activation= 'relu'),
        tf.keras.layers.Dense(units= 10, kernel_initializer= 'variance_scaling', activation= 'softmax')
    ])
    model.compile(optimizer = 'adam', 
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                  metrics = ['accuracy'])
    return model

In [ ]:
model = create_model()

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=30)

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
def modelpredict(model, gray28s, print=False):
    p=model(gray28s)
    if print:
        print(p)
    return np.argmax(p, axis=-1)

In [ ]:
start = 1000
v = modelpredict(model, x_test[start:(start+10)])
for i in range(start,start+10):
    plt.imshow((x_test[i]*255).astype('uint8'), cmap=plt.cm.gray)
    print(y_test[i], v[i-start])
    plt.show()

In [ ]:
def modelpredictfile(model,imgname):
    myimg = cv2.imread(imgname, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(myimg, cv2.COLOR_BGRA2GRAY)
    gray28 = cv2.resize(gray, (28,28), interpolation = cv2.INTER_AREA)
    gray28 = np.reshape(gray28, (28,28,1))
    d = modelpredict(model, np.array([gray28/255.0]))[0]
    print(imgname, d)

In [ ]:
import cv2

In [ ]:
for name in ["b-eight.png","b-five-fat.png","b-four-1.png","b-four-fat.png","b-four.png","b-nine-fat-2.png","b-nine-fat-3.png",
    "b-nine-fat-4.png","b-nine-fat-5.png","b-nine-fat.png","b-one-fat.png","b-seven-fat-2.png","b-seven-fat-3.png",
    "b-seven-fat-4.png","b-seven-fat-5.png","b-seven-fat.png","b-six-1.png","b-six-2.png","b-six-fat.png","b-six.png",
    "b-three-fat.png","b-three.png","b-two-fat.png","b-zero-fat.png"]:
    modelpredictfile(model, name)


In [ ]:
import tensorflowjs as tfjs

In [ ]:
tfjs.converters.save_keras_model(model, "Saved_Model/digit_cnn_model")